In [ ]:
import math

import pandas as pd
import networkx as nx

In [ ]:
# Load the records table with cleaned up country names as the primary data frame
records_path = "../data/records-renamed.csv"
df = pd.read_csv(records_path, index_col='ID', keep_default_na=False)

## Helpers

In [ ]:
# Helper constants
GA = 'General Assembly'
SC = 'Security Council'
START = 1946
END = 2024

In [ ]:
# Filter DFs for different parameters

def date_is_between(date, start_year, end_year):
    return date.str[0:4].astype(int).between(start_year, end_year)

def filter_year(df, start_year, end_year):
    return df[date_is_between(df['Date'], start_year, end_year)]

def filter_body(df, body):
    return df[df['Body'] == body]

def filter(df, body, start_year, end_year):
    return df[(df['Body'] == body) & (date_is_between(df['Date'], start_year, end_year))]

## Connect Countries

In [ ]:
vote_weights = {
    'Y': {
        'Y': 1.0,
        'N': 0.0,
        'A': 0.5,
        'X': 0.5
    },
    'N': {
        'Y': 0.0,
        'N': 1.0,
        'A': 0.5,
        'X': 0.5
    },
    'A': {
        'Y': 0.5,
        'N': 0.5,
        'A': 1.0,
        'X': 1.0
    },
    'X': {
        'Y': 0.5,
        'N': 0.5,
        'A': 1.0,
        'X': 1.0
    }
}

In [ ]:
def connect_countries(net, country1, country2, vote1, vote2):
    vote_weight = vote_weights[vote1][vote2]
    if net.has_edge(country1, country2):
        total = net[country1][country2]['total']
        points = net[country1][country2]['points']
        
        net[country1][country2]['total'] = total + 1
        net[country1][country2]['points'] = points + vote_weight
    else:
        net.add_edge(country1, country2, points = vote_weight, total=1)
        
def connect_voting_points(net, voting_point_1, voting_point_2):
    vote_1 = voting_point_1[0]
    vote_2 = voting_point_2[0]

    country_1 = voting_point_1[2:]
    country_2 = voting_point_2[2:]

    connect_countries(net, country_1, country_2, vote_1, vote_2)

In [ ]:
# Connect the countries and calculate their total agreement
def print_progress(finished, total, next_to_print):
    progress = math.floor(finished * 100 / total)
    if progress >= next_to_print:
        print('Progress: {}%'.format(progress))
        return next_to_print + 5
    else:
        return next_to_print

def calculate_agreement(net):
    for countries in net.edges:
        edge = net[countries[0]][countries[1]]
        edge['agreement'] = edge['points'] * 100 / edge['total']

def connect_all_countries(net, body, start_year, end_year):
    df_wip = df[(df['Voting Data'] != 'Concensus') & (df['Voting Data'] != 'N/A')]
    df_wip = filter(df_wip, body, start_year, end_year)

    finished = 0
    next_to_print = 0
    total = df_wip.shape[0]

    for index, row in df_wip.iterrows():
        voting_points = row['Voting Data'].split(';')
        for i in range(0, len(voting_points)):
            for j in range(i + 1, len(voting_points)):
                connect_voting_points(net, voting_points[i], voting_points[j])

        finished += 1
        next_to_print = print_progress(finished, total, next_to_print)

    calculate_agreement(net)

## Create Graphs

In [ ]:
country_coord_path = "./country-coord.csv"
country_coords = pd.read_csv(country_coord_path, index_col='Country')
country_coords.index= country_coords.index.str.upper()

In [ ]:
def add_coordinates(net):
    for country in net:
        if country in country_coords.index:
            country_coord = country_coords.loc[country]
            net.nodes[country]['x'] = country_coord['Longitude (average)']
            net.nodes[country]['y'] = country_coord['Latitude (average)']
        else:
            print('No coords found for ' + country)

In [ ]:
def create_graph(name, body, start_year, end_year):
    net = nx.Graph()
    
    connect_all_countries(net, body, start_year, end_year)
    add_coordinates(net)
    
    output_path = "../graphs/{}.gml".format(name)
    nx.write_gml(net, output_path)
    
    return net

In [ ]:
#net_ga_all =  create_graph('net_ga_all', GA, START, END)
#net_sc_all =  create_graph('net_sc_all', SC, START, END)
#net_ga_2006_end = create_graph('net_ga_2006_end', GA, 2006, END)

## Analyze Graph

In [ ]:
# Select the graph to analaze
significant_total = 0
net = net_sc_all

In [ ]:
print('Total countries: ' + str(len(net.nodes)))

In [ ]:
def friendly_edge_to_string(edge, country_buffer_1, country_buffer_2):
    country1 = edge[0].ljust(country_buffer_1)
    country2 = edge[1].ljust(country_buffer_2)
    
    total = str(edge[2]['total']).ljust(5)
    points = str(edge[2]['points']).rjust(6)
    agreement = round(edge[2]['agreement'], 1)
    
    return '{}, {} - {}/{} ({}%)'.format(country1, country2, points, total, agreement)

def edge_country_1_length(edge):
    return len(edge[0])

def edge_country_2_length(edge):
    return len(edge[1])

def edge_country_2_length(edge):
    return len(edge[1])

def friendly_edge_print(edges):
    max_cnt_1_length = max(map(edge_country_1_length, edges))
    max_cnt_2_length = max(map(edge_country_2_length, edges))
    for edge_str in map(lambda e: friendly_edge_to_string(e, max_cnt_1_length, max_cnt_2_length), edges):
        print(edge_str)

In [ ]:
# Sort edges by agreement
sorted_edges = sorted(net.edges(data=True), key=lambda edge: edge[2]['agreement'])

In [ ]:
def has_significant_total(edge):
    return edge[2]['total'] >= significant_total

sorted_edges =  list(__builtin__.filter(has_significant_total, sorted_edges))

In [ ]:
# Show countries with the least agreement
friendly_edge_print(sorted_edges[:5])

In [ ]:
# Show countries with the most agreement
friendly_edge_print(sorted_edges[-5:])